In [ ]:
#ЗАВДАННЯ 1. ПРОГНОЗ СЛІВ
import os
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

# Текст, який ви хочете використовувати
texts = """
Думи мої, думи мої,
Лихо мені з вами!
Нащо стали на папері
Сумними рядами?..
Чом вас вітер не розвіяв
В степу, як пилину?
Чом вас лихо не приспало,
Як свою дитину?..

Бо вас лихо на світ на сміх породило,
Поливали сльози... чом не затопили,
Не винесли в море, не розмили в полі?.
Не питали б люде, що в мене болить,
Не питали б, за що проклинаю долю,
Чого нуджу світом? "Нічого робить", —
Не сказали б на сміх...

"""

texts = texts.replace('\ufeff', '')  # Прибираємо перший невидимий символ

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&;()*+,-./:<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([texts])
res = to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)


def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # Дописуємо рядок

    return res


res = buildPhrase("Думи мої думи")
print(res)


[('думи', 2), ('мої', 2), ('лихо', 3), ('мені', 1), ('з', 1), ('вами', 1), ('нащо', 1), ('стали', 1), ('на', 4), ('папері', 1)]
(77, 1000)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_1 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
3/3 [==============================] - 2s 18ms/step - loss: 6.9086 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 6.8587 - accuracy: 0.0811
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 6.8133

In [ ]:
#Завдання 2. Прогнозування слів з Embedding

import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

# Текст, який ви хочете використовувати
texts = """
Думи мої, думи мої,
Лихо мені з вами!
Нащо стали на папері
Сумними рядами?..
Чом вас вітер не розвіяв
В степу, як пилину?
Чом вас лихо не приспало,
Як свою дитину?..

Бо вас лихо на світ на сміх породило,
Поливали сльози... чом не затопили,
Не винесли в море, не розмили в полі?.
Не питали б люде, що в мене болить,
Не питали б, за що проклинаю долю,
Чого нуджу світом? "Нічого робить", —
Не сказали б на сміх...

"""
texts = texts.replace('\ufeff', '')  # Прибираємо перший невид. символ

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([texts])

res = np.array( data[0] )
print(res.shape)

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)


def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


res = buildPhrase("Думи мої думи")
print(res)

[('думи', 2), ('мої', 2), ('лихо', 3), ('мені', 1), ('з', 1), ('вами', 1), ('нащо', 1), ('стали', 1), ('на', 4), ('папері', 1)]
(77,)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_3 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434280 (1.66 MB)
Trainable params: 434280 (1.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
3/3 [==============================] - 2s 13ms/step - loss: 6.9082 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================]

In [ ]:
#Завдання 3. Побудова моделі RNN
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

# Текст, який ви хочете використовувати
texts = """
Думи мої, думи мої,
Лихо мені з вами!
Нащо стали на папері
Сумними рядами?..
Чом вас вітер не розвіяв
В степу, як пилину?
Чом вас лихо не приспало,
Як свою дитину?..

Бо вас лихо на світ на сміх породило,
Поливали сльози... чом не затопили,
Не винесли в море, не розмили в полі?.
Не питали б люде, що в мене болить,
Не питали б, за що проклинаю долю,
Чого нуджу світом? "Нічого робить", —
Не сказали б на сміх...

"""
texts = texts.replace('\ufeff', '')  # Прибираємо перший невид. символ

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

data = tokenizer.texts_to_sequences([texts])

res = np.array( data[0] )
print(res.shape)

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh', return_sequences=True))
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)


def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


res = buildPhrase("Думи мої думи")
print(res)

[('думи', 2), ('мої', 2), ('лихо', 3), ('мені', 1), ('з', 1), ('вами', 1), ('нащо', 1), ('стали', 1), ('на', 4), ('папері', 1)]
(77,)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 3, 128)            128000    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 3, 128)            32896     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_4 (Dense)             (None, 1000)              65000     
                                                                 
Total params: 238248 (930.66 KB)
Trainable params: 238248 (930.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
3

In [ ]:
#ЗАВДАННЯ. Сентимент-Аналіз
import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Задані тексти
texts_true = [
    "Це текст, який містить позитивну інформацію.",
    "Ще один приклад позитивного тексту.",
    "Тексти можуть містити різні емоції, включаючи позитивні."
]

texts_false = [
    "Це текст, який містить негативну інформацію.",
    "Не всі тексти будуть позитивними.",
    "Можна зустріти і негативні тексти."
]

# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

t = "Я люблю позитивний настрій".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data[0]))

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')


3 3 6
[('це', 2), ('текст', 2), ('який', 2), ('містить', 2), ('позитивну', 1), ('інформацію', 2), ('ще', 1), ('один', 1), ('приклад', 1), ('позитивного', 1)]
Це текст, який містить позитивну інформацію.
[[ 0  0  0  0  2  3  4  5  7  6]
 [ 0  0  0  0  0  8  9 10 11 12]
 [ 0  0  0  1 13 14 15 16 17 18]
 [ 0  0  0  0  2  3  4  5 19  6]
 [ 0  0  0  0  0 20 21  1 22 23]
 [ 0  0  0  0  0 24 25 26 27  1]]
[('тексти', 1), ('це', 2), ('текст', 3), ('який', 4), ('містить', 5), ('інформацію', 6), ('позитивну', 7), ('ще', 8), ('один', 9), ('приклад', 10), ('позитивного', 11), ('тексту', 12), ('можуть', 13), ('містити', 14), ('різні', 15), ('емоції', 16), ('включаючи', 17), ('позитивні', 18), ('негативну', 19), ('не', 20), ('всі', 21), ('будуть', 22), ('позитивними', 23), ('можна', 24), ('зустріти', 25), ('і', 26), ('негативні', 27)]
(6, 10) (6, 2)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #  

In [ ]:
# Визначення результату
if np.argmax(res) == 0:
    print("Це негативний текст.")
else:
    print("Це позитивний  текст.")

Це позитивний  текст.
